In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,date
from pandas.tseries.holiday import USFederalHolidayCalendar
import os

In [2]:
base_path_DPMU = "..\\..\\data\\summer\\targets\\trimmed_targets"
all_months_DPMU = {} ##{mm:{a:, b:, c:}, mm:{a:,b:,c:}}

include_kmeans = 1
include_PCA = 0
include_holidays = 0
include_wknds = 0
include_dayofweek = 0
include_phase = 0
%store include_kmeans
%store include_PCA
%store include_holidays
%store include_wknds
%store include_dayofweek
%store include_phase

Stored 'include_kmeans' (int)
Stored 'include_PCA' (int)
Stored 'include_holidays' (int)
Stored 'include_wknds' (int)
Stored 'include_dayofweek' (int)
Stored 'include_phase' (int)


# Input (DPMU) Preprocessing (sends to X.csv)

In [3]:
#modifies raw (trimmed) csv to add time features and process the various phases/houses
def modify_csv(month_phase_csv, month_func, year_user):
    ##time modifications
    month_fill = 0
    if month_func == 'june':
        month_fill = 6
    elif month_func == 'july':
        month_fill = 7
    elif month_func == 'aug':
        month_fill = 8
    elif month_func == 'sept':
        month_fill = 9
    time = month_phase_csv['# timestamp']
    time = pd.Series(time)
    time = time.str.split(" ")
    for index in range(len(time)):
        del time[index][2]
    time
    month = []
    hour = []
    for index in range(len(time)):
        month.append(datetime.strptime(time[index][0],'%Y-%m-%d'))
        hour.append(datetime.strptime(time[index][1],'%H:%M:%S'))
    date = []
    timestamp = []

    for index in range(len(time)):
        for jindex in range(len(time[index])):
            if "-" in time[index][jindex]:
                date.append(time[index][jindex].split("-"))
            if ":" in time[index][jindex]:
                timestamp.append(time[index][jindex].split(":"))
    date = np.array(date)
    timestamp = np.array(timestamp)
    #separating the individual values into different list
    year = []
    month = []
    day = []
    for y in range(len(date)):
        year.append(date[y][0])
        month.append(date[y][1])
        day.append(date[y][2])

    #do special processing for the hours and minutes
    day_min = []
    for y in range(len(timestamp)):
        hour_min_added = int(timestamp[y][0]) * 60
        hour_min_added = hour_min_added + int(timestamp[y][1])
        day_min.append(hour_min_added) 

    ##code to overwrite 'January' outputs for csv's    
    year = [year_user] * len(date)
    month = [month_fill] * len(date)
        
    time_df = pd.DataFrame()
    time_df["year"] = year
    time_df["month"] = month
    time_df["day"] = day
    

    time_df["hour_min"] = day_min
        
    month_phase_mod = month_phase_csv.drop(['Meter5', '# timestamp'], axis=1)
    month_phase_mod = pd.concat([month_phase_mod,time_df], axis=1)
    #print (month_phase_mod)
    return month_phase_mod

In [4]:
#modifies the global all_months_DPMU. When this is over, that array will have all the necessary information to be concatenated into a giant array.
def sep_DPMU_features():
    #print (base_path_DPMU)
    for month in os.listdir(base_path_DPMU):
        month_path = os.path.join(base_path_DPMU,month)
        month_dict = {}
        for multi_voltage_output in os.listdir(month_path):
            full_csv_path = os.path.join(month_path,multi_voltage_output)
            #print(full_csv_path)
            cut_voltage = multi_voltage_output[:-4]
            #print(cut_voltage)
            letter = cut_voltage[-1:]
            #print(letter)
            dpmu_phase_month = "dpmu_phase" + letter + "_" + month
            raw_csv = pd.read_csv(full_csv_path)
            month_dict[letter] = modify_csv(raw_csv, month, 2018)
        #add month dictionary
        all_months_DPMU[month]=month_dict
    #return all the months it found in the base dir
    return list(all_months_DPMU.keys())
    

In [5]:
def DPMU(phase, dpmu_data, letter):
    phase_letter = ord(letter) - 65
    #print(phase_letter)
    for (columnName, columnData) in phase.iteritems():
        i=0
        if 'Meter' in columnName: ##meters
            split = columnName.split('Meter')
            house_name = f'Meter{letter}_'+ split[1]
            i+=1
            dpmu_values = pd.Series(columnData.values).str.replace("i", "j").apply(lambda x: abs(complex(x)))
            dpmu_data[house_name] = dpmu_values
        elif 'node' in columnName : ##nodes
            split_column_names = columnName.split('_')
            house_name = 'house'+str(split_column_names[phase_letter])
            dpmu_values = pd.Series(columnData.values).str.replace("i", "j").apply(lambda x: abs(complex(x)))
            dpmu_data[house_name] = dpmu_values
        else: ##timestamps, day_min
            if letter == 'A':
                dpmu_data[columnName] = pd.Series(columnData.values)
    return dpmu_data
    

In [6]:
def concat_all_DPMU():
    all_months_to_concat = []
    for month in all_months_DPMU:
        #print(month)
        dpmu_data_month = pd.DataFrame()
        for phase in all_months_DPMU[month]:
            #print (phase)
            dpmu_data_month = DPMU(all_months_DPMU[month][phase.upper()], dpmu_data_month, phase.upper())
        all_months_to_concat.append(dpmu_data_month)
    input_dataframe_DPMU = pd.DataFrame() ##concatenates all months, directly to input
    input_dataframe_DPMU = pd.concat(all_months_to_concat, ignore_index = True)
    return (input_dataframe_DPMU)
    
        

In [7]:
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
def sep_DPMU_clusters_kmeans(): #simoni
    # determined convergence to be 5 clusters .. see png
    #sum_of_squared_distances = []
    #K = range(1,37)
    #for k in K:
    #    print(k)
    #    k_means = KMeans(n_clusters=k)
    #    model = k_means.fit(in_csv)
    #    sum_of_squared_distances.append(k_means.inertia_)
    #    print(k_means.inertia_)
    #plt.plot(K, sum_of_squared_distances, 'bx-')
    #plt.xlabel('k')
    #plt.ylabel('sum_of_squared_distances')
    #plt.title('elbow method for optimal k')
    #plt.show()
    kmeans_df = in_csv
    k_means = KMeans(n_clusters=5)
    model = k_means.fit(kmeans_df)
    #labels = k_means.labels_
    #print(metrics.silhouette_score(kmeans_df, labels, metric = 'euclidean'))
    #print(metrics.calinski_harabasz_score(kmeans_df, labels))

In [8]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
def sep_DPMU_clusters_PCA(): #simoni
    pca_df = in_csv
    pca = PCA(n_components=37) # set to max components to determine convergence
    pca.fit(pca_df)
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Number of components')
    plt.ylabel('Cumulative explained variance') # returns that best # of components for convergence is ~10

In [9]:
def phase_clustering(): #swetha
    print(in_csv)
    #only keep one house/meter from each phase
    in_csv.drop(in_csv.columns.difference(['MeterA_1', 'MeterB_1', 'MeterC_1', 'housenode1', 'housenode2', 'housenode3', 'year',
                                          'month', 'day', 'hour_min']), 1, inplace=True)
    #print phase_clustering_df

    

In [10]:
#return df modified to include flags for holidays
def add_holidays(): #swetha
    print(in_csv)
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start='2018-01-01', end='2018-12-31').to_pydatetime()

    holiday_array = []
    for i in range((in_csv.shape[0])):
        if datetime(int(in_csv.loc[in_csv.index[i], 'year']),
                   int(in_csv.loc[in_csv.index[i], 'month']),
                   int(in_csv.loc[in_csv.index[i], 'day'])) in holidays:
            holiday_array.append(1)
        else:
            holiday_array.append(0)
            
    in_csv["is_holiday"] = holiday_array


In [11]:
#return df modified to include flags for weekend
def add_weekends(): #swetha

    wknd_array = []
    for i in range((in_csv.shape[0])):
        wkday = date(int(in_csv.loc[in_csv.index[i], 'year']),
                   int(in_csv.loc[in_csv.index[i], 'month']),
                   int(in_csv.loc[in_csv.index[i], 'day'])).weekday()
        is_wknd = 1 if wkday>=4 else 0
        wknd_array.append(is_wknd)
    in_csv["is_weekend"] = wknd_array


In [12]:
#return df modified to include flags for weekend
def add_dayofweek(): #swetha

    wk_array = []
    for i in range((in_csv.shape[0])):
        wkday = date(int(in_csv.loc[in_csv.index[i], 'year']),
                   int(in_csv.loc[in_csv.index[i], 'month']),
                   int(in_csv.loc[in_csv.index[i], 'day'])).isoweekday()
        wk_array.append(wkday)

    in_csv["day_of_week"] = wk_array


In [ ]:
##all_months_DPMU will have a dictionary of csv's for each month, each phase
sep_DPMU_features()

In [ ]:
##input_dataframe_DPMU will have large dataframe of input data
send_input_to_csv = pd.DataFrame()
send_input_to_csv = concat_all_DPMU()

In [ ]:
send_input_to_csv

In [ ]:
list(send_input_to_csv.columns)

In [ ]:
send_input_to_csv = send_input_to_csv[['MeterA_1', 'MeterA_2', 'MeterA_3', 
                                       'MeterB_1', 'MeterB_2', 'MeterB_3',
                                      'MeterC_1', 'MeterC_2', 'MeterC_3',
                                      'housenode1','housenode4','housenode7',
                                       'housenode13','housenode10','housenode22',
                                       'housenode19','housenode16',
                                      'house2','house5', 'house8','house14',
                                       'house11','house23','house20','house17',
                                       'house3','house6','house9','house15','house12',
                                       'house24','house21','house18',
                                       'year','month','day','hour_min']]

# No clustering

In [ ]:
in_csv = pd.DataFrame()
in_csv = send_input_to_csv.copy()
send_input_to_csv.to_csv('../../data/input/X.csv')

In [ ]:
in_csv

# K-means, PCA, phase, holidays, weekends

In [ ]:
csv_name = "X"


if (include_kmeans):
    sep_DPMU_clusters_kmeans()
    csv_name = csv_name + "_kmeans"

if(include_PCA):
    sep_DPMU_clusters_PCA()
    csv_name = csv_name + "_pca"

if(include_phase):
    phase_clustering()
    csv_name = csv_name + "_phase"
    
if(include_holidays):
    add_holidays()
    csv_name = csv_name + "_holidays"
    
if(include_wknds):
    add_weekends()
    csv_name = csv_name + "_weekends"
    
if(include_dayofweek):
    add_dayofweek()
    csv_name = csv_name + "_weekdays"


In [ ]:
csv_name

In [ ]:
in_csv

In [ ]:
csv_name = csv_name + ".csv"
in_csv.to_csv('../../data/input/' + csv_name)

# Output (Grid) Preprocessing (sends to Y.csv)

In [ ]:
base_path_Player = "..\\..\\data\\summer\\input_csv"

In [ ]:
def set_output():
    month_players = []
    for month in os.listdir(base_path_Player):
        month_path = os.path.join(base_path_Player,month)
        input_player_df = pd.DataFrame()
        for filename in os.listdir(month_path):
            if str(filename).startswith('25.csv'):
                continue
            player = pd.read_csv(os.path.join(month_path, filename))
            player['grid'] = player['grid'].str.replace("i", "j").apply(lambda x: complex(x))
            node_id = player['id'].loc[0]
            column_name_p = 'p_'+str(node_id)
            column_name_q = 'q_'+str(node_id)
        #change .head() value to target dataframe row number
            input_player_df[column_name_p] = player['grid'].apply(lambda x: x.real).head(44640)
            input_player_df[column_name_q] = player['grid'].apply(lambda x: x.imag).head(44640)
        month_players.append(input_player_df)
    output_dataframe_Player = pd.DataFrame() ##concatenates all months, directly to input
    output_dataframe_Player = pd.concat(month_players, ignore_index = True)
    return (output_dataframe_Player)
    
    

In [ ]:
##input_dataframe_DPMU will have large dataframe of input data
send_targets_to_csv = pd.DataFrame()
send_targets_to_csv = set_output()

In [ ]:
send_targets_to_csv

In [ ]:
send_targets_to_csv.to_csv('../../data/outputs/Y.csv')